<a href="https://colab.research.google.com/github/farout101/random-codes/blob/main/%5BDevFest2025%5D_ADK_Learning_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Welcome to Your ADK Adventure - Tools & Memory! 🚀

Welcome, Agent Architect! This notebook is your guide to giving your AI agents two essential superpowers: custom tools and conversational memory.

By the end of this adventure, you will be able to:

- **Build a Foundational Agent**: Create a simple but effective AI agent from scratch using the Google Agent Development Kit (ADK).

- **Grant New Skills with Custom Tools**: Teach an agent to perform new tasks by connecting it to external APIs, like a real-time weather service.

- **Create a Team of Agents**: Assemble a multi-agent system where a primary agent can delegate specialized tasks to other agents.

- **Master Conversational Memory**: Understand the critical role of Sessions in enabling agents to remember previous interactions, handle feedback, and carry on a coherent conversation.


Let's get this adventure started!

## Author

HI, I'm Qingyue (Annie) Wang, a developer advocate and AI engineer at **Google**, passionate about helping developers build with AI and cloud technologies :)


If you have questions with this notebook, contact me on [LinkedIn](https://www.linkedin.com/in/qingyuewang/) , [X](https://twitter.com/qingyuewang) or email anniewangtech0510@Gmail.com


```
  (\__/)
  (•ㅅ•)
  /づ  📚      Enjoy learning AI Agents :)
```


-------------
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

👉 **Get Your API Key HERE**: https://codelabs.developers.google.com/onramp/instructions#1

 -----------------------------------------------------------------------------

```
 ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️
   /\_/\     /\_/\     /\_/\      /\_/\       /\_/\
  ( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )    ( o_o )             
```


## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries and securely configures your Google API key so your agents can access the power of Gemini.

In [5]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [6]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


---
## Part 1: Your First Agent - The Day Trip Genie 🧞

Meet your first creation! The `day_trip_agent` is a simple but powerful assistant. We're making it a little smarter by teaching it to understand **budget constraints**.

* **Agent**: The brain of the operation, defined by its instructions, tools, and the AI model it uses.
* **Session**: The conversation history. For this simple agent, it's just a container for a single request-response.
* **Runner**: The engine that connects the `Agent` and the `Session` to process your request and get a response.

```
+--------------------------------------------------+
|         Spontaneous Day Trip Agent 🤖            |
|--------------------------------------------------|
|  Model: gemini-2.5-flash                         |
|  Description:                                    |
|   Generates full-day trip itineraries based on   |
|   mood, interests, and budget                    |
|--------------------------------------------------|
|  🔧 Tools:                                       |
|   - Google Search                                |
|--------------------------------------------------|
|  🧠 Capabilities:                                |
|   - Budget Awareness (cheap / splurge)           |
|   - Mood Matching (adventurous, relaxing, etc.)  |
|   - Real-Time Info (hours, events)               |
|   - Morning / Afternoon / Evening plan           |
+--------------------------------------------------+

            ▲
            |
    +------------------+
    |   User Input     |
    |------------------|
    |  Mood            |
    |  Interests       |
    |  Budget          |
    +------------------+

            |
            ▼

+--------------------------------------------------+
|             Output: Markdown Itinerary           |
|--------------------------------------------------|
| - Time blocks (Morning / Afternoon / Evening)    |
| - Venue names with links and hours               |
| - Budget-matching activities                     |
+--------------------------------------------------+
```


In [7]:
# --- Agent Definition ---

def create_day_trip_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
        instruction="""
        You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

        Your Mission:
        Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

        Guidelines:
        1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
        2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
        3. **Real-Time Focus**: Search for current operating hours and special events.
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
        """,
        tools=[google_search]
    )

day_trip_agent = create_day_trip_agent()
print(f"🧞 Agent '{day_trip_agent.name}' is created and ready for adventure!")

🧞 Agent 'day_trip_agent' is created and ready for adventure!


In [8]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [18]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=day_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a relaxing and artsy day trip near Mandalay. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(day_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

🗣️ User Query: 'Plan a relaxing and artsy day trip near Mandalay. Keep it affordable!'

🚀 Running query for agent: 'day_trip_agent' in session: '1b2904ee-4316-4285-a983-7aa3cb56d9a9'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Here's a relaxing and artsy day trip near Mandalay, designed to be affordable and immerse you in local culture and craftsmanship. This itinerary focuses on the ancient cities of Amarapura and Sagaing, easily accessible from Mandalay.

---

## Relaxing & Artsy Day Trip near Mandalay (Affordable)

**Budget Note:** This itinerary prioritizes free or low-cost activities. Transportation will be the main variable cost; using shared taxis, local pick-up trucks, or motorbike taxis will keep it affordable. Budget approximately 15,000-25,000 Kyats (approx. $7-$12 USD) for transport and 10,000 Kyats ($5 USD) for the Mahamuni Buddha Temple entrance fee, plus food.

---

### **Morning (9:00 AM - 12:30 PM): Mandalay's Artistic Heritage**

*   **9:00 AM - 10:00

Here's a relaxing and artsy day trip near Mandalay, designed to be affordable and immerse you in local culture and craftsmanship. This itinerary focuses on the ancient cities of Amarapura and Sagaing, easily accessible from Mandalay.

---

## Relaxing & Artsy Day Trip near Mandalay (Affordable)

**Budget Note:** This itinerary prioritizes free or low-cost activities. Transportation will be the main variable cost; using shared taxis, local pick-up trucks, or motorbike taxis will keep it affordable. Budget approximately 15,000-25,000 Kyats (approx. $7-$12 USD) for transport and 10,000 Kyats ($5 USD) for the Mahamuni Buddha Temple entrance fee, plus food.

---

### **Morning (9:00 AM - 12:30 PM): Mandalay's Artistic Heritage**

*   **9:00 AM - 10:00 AM: King Galon Gold Leaf Workshop**
    Begin your day by witnessing the intricate and ancient art of gold leaf making. At workshops like King Galon, you can observe skilled artisans meticulously pounding gold into wafer-thin sheets by hand. The entrance is free, and staff are usually happy to explain the laborious process. You'll gain an appreciation for the gold leaf that adorns many pagodas in Myanmar.
    *   *Opening Hours:* Daily, 7:30 AM - 6:00 PM.
    *   *Cost:* Free to observe (donations welcome).
    *   *Location:* Various locations in Mandalay, King Galon is well-known.

*   **10:15 AM - 11:45 AM: Mahamuni Buddha Temple**
    Visit one of Myanmar's most important pilgrimage sites. The Mahamuni Buddha image is highly revered, and devout Buddhists continuously apply gold leaf to it. Observing the devotion and the art within the temple offers a profound cultural experience. Men can even participate in applying gold leaf. The temple complex also houses ancient bronze statues and a small market selling religious items and traditional puppets.
    *   *Opening Hours:* Daily, 6:00 AM - 8:00 PM.
    *   *Cost:* 10,000 Kyats (approx. $5 USD) for foreign visitors.
    *   *Location:* Between 82nd and 84th Streets, Mandalay.
    *   *Travel:* Take a local rickshaw (approx. $2 USD from downtown) or shared taxi.

*   **12:00 PM - 1:00 PM: Shwe In Bin Monastery**
    Before leaving Mandalay, seek out the serene Shwe In Bin Monastery. This exquisite teak wood monastery, built in the late 19th century, is renowned for its intricate carvings and traditional Burmese architecture. It offers a peaceful atmosphere away from the crowds and is free to enter.
    *   *Opening Hours:* Generally open during daylight hours, but specific times can vary.
    *   *Cost:* Free entry.
    *   *Location:* Corner of 89th and 38th Streets, Mandalay.
    *   *Travel:* A short rickshaw or motorbike taxi ride from Mahamuni.

### **Afternoon (1:00 PM - 5:00 PM): Monastic Life & Scenic Views in Amarapura & Sagaing**

*   **1:00 PM - 1:30 PM: Travel to Amarapura**
    Take a shared taxi or a local pick-up truck from Mandalay to Amarapura, a former royal capital. This is a very affordable way to travel between the cities.

*   **1:30 PM - 2:30 PM: Mahagandayon Monastery**
    Visit one of the largest teaching monasteries in Myanmar. While the famous alms procession for thousands of monks is typically earlier in the morning (around 10:00-10:30 AM), it's still possible to observe monastic life and enjoy the tranquil grounds in the early afternoon.
    *   *Cost:* Free to visit.
    *   *Location:* Amarapura.

*   **2:30 PM - 3:30 PM: Affordable Local Lunch**
    Enjoy a simple, delicious, and affordable Burmese lunch at a local tea shop or restaurant in Amarapura.

*   **3:30 PM - 5:00 PM: Sagaing Hill Exploration**
    Head to Sagaing Hill, a renowned Buddhist center dotted with hundreds of white, silver, and gold pagodas and monasteries. You can hire a motorbike taxi for a few hours or, for the truly adventurous and budget-conscious, rent a bicycle (approx. 1500 Kyats/day). Explore at your leisure, focusing on the panoramic views from pagodas like Soon U Ponya Shin Pagoda. The peaceful atmosphere and stunning scenery offer a relaxing escape.
    *   *Opening Hours:* Many pagodas are open from early morning to late afternoon. Sagaing Hill itself is accessible from 7:00 AM - 7:30 PM.
    *   *Cost:* Many smaller pagodas are free. There might be a zone fee for the Sagaing-Mingun area, but often free exploration of the hill is possible.
    *   *Location:* Across the Irrawaddy River from Amarapura.

### **Evening (5:00 PM - 7:00 PM): Sunset at U Bein Bridge**

*   **5:00 PM - 5:30 PM: Return to Amarapura & U Bein Bridge**
    Travel back to Amarapura to reach U Bein Bridge before sunset.

*   **5:30 PM - 7:00 PM: Sunset at U Bein Bridge**
    Conclude your day at the iconic U Bein Bridge, the longest teakwood bridge in the world. Take a leisurely walk across the 1.2-kilometer bridge and soak in the serene atmosphere as the sun dips below the horizon, casting beautiful colors over Taungthaman Lake. This is a perfect relaxing and photogenic end to your day. Sunset in November is around 5:25 PM.
    *   *Cost:* Free.
    *   *Location:* Amarapura.

*   **7:00 PM onwards: Dinner in Amarapura or Return to Mandalay**
    Enjoy an affordable dinner at a local restaurant near U Bein Bridge or head back to Mandalay for more dining options.

---

--------------------------------------------------



---
## Part 2: Supercharging Agents with Custom Tools 🛠️

So far, we've used the powerful built-in `GoogleSearch` tool. But the true power of agents comes from connecting them to your own logic and data sources.

This is where **custom tools** come in. Let's explore three patterns for giving your agent new skills, using real-world, practical examples.

### 2.1 The Simple `FunctionTool`: Calling a Real-Time Weather API

The most direct way to create a tool is by writing a Python function. This is perfect for synchronous tasks like fetching data from an API.

**Key Concept:** The function's **docstring** is critical. The ADK uses it as the tool's official description, which the LLM reads to understand its purpose, parameters, and when to use it.

In this example, we'll create a tool that calls the **free, public U.S. National Weather Service API** to get a real-time forecast. No API key needed!

In [10]:
# --- Tool Definition: A function that calls a live public API ---
import requests
import json

# A simple lookup to avoid needing a separate geocoding API for this example
LOCATION_COORDINATES = {
    "sunnyvale": "37.3688,-122.0363",
    "san francisco": "37.7749,-122.4194",
    "lake tahoe": "39.0968,-120.0324"
}

def get_live_weather_forecast(location: str) -> dict:
    """Gets the current, real-time weather forecast for a specified location in the US.

    Args:
        location: The city name, e.g., "San Francisco".

    Returns:
        A dictionary containing the temperature and a detailed forecast.
    """
    print(f"🛠️ TOOL CALLED: get_live_weather_forecast(location='{location}')")

    # Find coordinates for the location
    normalized_location = location.lower()
    coords_str = None
    for key, val in LOCATION_COORDINATES.items():
        if key in normalized_location:
            coords_str = val
            break
    if not coords_str:
        return {"status": "error", "message": f"I don't have coordinates for {location}."}

    try:
        # NWS API requires 2 steps: 1. Get the forecast URL from the coordinates.
        points_url = f"https://api.weather.gov/points/{coords_str}"
        headers = {"User-Agent": "ADK Example Notebook"}
        points_response = requests.get(points_url, headers=headers)
        points_response.raise_for_status() # Raise an exception for bad status codes
        forecast_url = points_response.json()['properties']['forecast']

        # 2. Get the actual forecast from the URL.
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()

        # Extract the relevant forecast details
        current_period = forecast_response.json()['properties']['periods'][0]
        return {
            "status": "success",
            "temperature": f"{current_period['temperature']}°{current_period['temperatureUnit']}",
            "forecast": current_period['detailedForecast']
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Agent Definition: An agent that USES the new tool ---

weather_agent = Agent(
    name="weather_aware_planner",
    model="gemini-2.5-flash",
    description="A trip planner that checks the real-time weather before making suggestions.",
    instruction="You are a cautious trip planner. Before suggesting any outdoor activities, you MUST use the `get_live_weather_forecast` tool to check conditions. Incorporate the live weather details into your recommendation.",
    tools=[get_live_weather_forecast]
)

print(f"🌦️ Agent '{weather_agent.name}' is created and can now call a live weather API!")

🌦️ Agent 'weather_aware_planner' is created and can now call a live weather API!


In [11]:
# --- Let's test the Weather-Aware Planner ---

async def run_weather_planner_test():
    weather_session = await session_service.create_session(app_name=weather_agent.name, user_id=my_user_id)
    query = "I want to go hiking near Lake Tahoe, what's the weather like?"
    print(f"🗣️ User Query: '{query}'")
    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_planner_test()

🗣️ User Query: 'I want to go hiking near Lake Tahoe, what's the weather like?'

🚀 Running query for agent: 'weather_aware_planner' in session: 'c2d1dc5e-b9f1-4157-a995-5a77a34ce9cf'...


EVENT: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'location': 'Lake Tahoe'
        },
        id='adk-a1c662d3-d8f5-4850-a9ae-fe1f66d5db30',
        name='get_live_weather_forecast'
      ),
      thought_signature=b'\n\xcb\x02\x01\xd1\xed\x8ao\xa8\x86\xb0!t\x8e\xcd\xd4r\x83\x8e\x0c\xb6!t\x9f@\xfd5\x05\x1c\xad\xdc)l\x82\xa5R\x885\xb1}\x9c,\xf3\xe07\x90(]\xe6e\xa4\x00\x07V\x1c\xf0O\x118\xa5/\x81\x92\\\xbc{\x0b\xcd9\xcad$[\xe5\xd9\x1aQ\xfc\x8c\x86=\xbb\xefBi\xf7m\x99GP\xad\xcd\xfa\xa6A\xfa\xed...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=20,
  prompt_token_count=187,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=187
    ),
  ],
  thought

EVENT: content=Content(
  parts=[
    Part(
      text='The weather near Lake Tahoe is 42°F, mostly cloudy with a slight chance of rain (20% chance of precipitation), and an east wind around 5 mph. Given these conditions, it would be a good idea to bring a waterproof jacket and dress in layers if you plan on hiking.',
      thought_signature=b'\n\xcc\x04\x01\xd1\xed\x8ao\r\xc9"j=\xaa_\xef\xd2\xec\xfd0pC\xb5\xa1\xb6\xd82\nr\xd9\xe7SO\xbdg\x0c\x01\x19\x83C\x04\xa6\x9f"\x86k\xee=\x80\x11\x90\x965\xa2[;\\\xc8\x05[M,\xad}I\xa2\xb9\xd6\x0e\xed?]\x19B\x9f\x9e\xd7O_\x8e!_\xee\x0e*\xacz\xe3\xc9\x0f\xf8\xce\x00\xab\xc3\x92-...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=62,
  prompt_token_count=272,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<

The weather near Lake Tahoe is 42°F, mostly cloudy with a slight chance of rain (20% chance of precipitation), and an east wind around 5 mph. Given these conditions, it would be a good idea to bring a waterproof jacket and dress in layers if you plan on hiking.

--------------------------------------------------



## 2.2 The Agent-as-a-Tool: Consulting a Specialist 🧑‍🍳

Why build one agent that does everything when you can build a **team of specialist agents?** The **Agent-as-a-Tool** pattern allows one agent to delegate a task to another agent.

**Key Concept:** This is different from a sub-agent. When Agent A calls Agent B as a tool, Agent B's response is passed **back to Agent A**. Agent A then uses that information to form its own final response to the user. It's a powerful way to compose complex behaviors from simpler, focused, and reusable agents.

### How It Works

Our top-level agent, the `trip_data_concierge_agent`, acts as the **Orchestrator**. It has two tools at its disposal:

1.  `call_db_agent`: A function that internally calls our `db_agent` to fetch raw data.
2.  `call_concierge_agent`: A function that calls the `concierge_agent`.

The `concierge_agent` itself has a tool: the `food_critic_agent`.

The flow for a complex query is:

1.  **User** asks the `trip_data_concierge_agent` for a hotel and a nearby restaurant.
2.  The **Orchestrator** first calls `call_db_agent` to get hotel data.
3.  The data is saved in `tool_context.state`.
4.  The **Orchestrator** then calls `call_concierge_agent`, which retrieves the hotel data from the context.
5.  The `concierge_agent` receives the request and decides it needs to use its own tool, the `food_critic_agent`.
6.  The `food_critic_agent` provides a witty recommendation.
7.  The `concierge_agent` gets the critic's response and politely formats it.
8.  This final, polished response is returned to the **Orchestrator**, which presents it to the user.

                         +-----------------------------------------------------------+
                         |              🧭 Trip Data Concierge Agent                 |
                         |-----------------------------------------------------------|
                         |  Model: gemini-2.5-flash                                  |
                         |  Description:                                             |
                         |   Orchestrates database query and travel recommendation  |
                         |-----------------------------------------------------------|
                         |  🔧 Tools:                                                |
                         |   1. call_db_agent                                        |
                         |   2. call_concierge_agent                                 |
                         +-----------------------------------------------------------+
                                      /                                \
                                     /                                  \
                                    ▼                                    ▼
        +-------------------------------------------+    +---------------------------------------------+
        |            🔧 Tool: call_db_agent         |    |         🔧 Tool: call_concierge_agent        |
        |-------------------------------------------|    |---------------------------------------------|
        | Calls: db_agent                           |    | Calls: concierge_agent                       |
        |                                           |    | Uses data from db_agent for recommendations |
        +-------------------------------------------+    +---------------------------------------------+
                                |                                          |
                                ▼                                          ▼
       +--------------------------------------------+   +------------------------------------------------+
       |              📦 db_agent                   |   |             🤵 concierge_agent                  |
       |--------------------------------------------|   |------------------------------------------------|
       | Model: gemini-2.5-flash                    |   | Model: gemini-2.5-flash                         |
       | Role: Return mock JSON hotel data          |   | Role: Hotel staff that handles user Q&A        |
       +--------------------------------------------+   | Tools:                                          |
                                                         |  - food_critic_agent                           |
                                                         +------------------------------------------------+
                                                                                 |
                                                                                 ▼
                                                       +------------------------------------------------+
                                                       |          🍽️ food_critic_agent                  |
                                                       |------------------------------------------------|
                                                       | Model: gemini-2.5-flash                         |
                                                       | Role: Gives a witty restaurant recommendation   |
                                                       +------------------------------------------------+


In [12]:
import asyncio
from google.adk.tools import ToolContext
from google.adk.tools.agent_tool import AgentTool

# Assume 'db_agent' is a pre-defined NL2SQL Agent
# For this example, we'll create placeholder agents.

db_agent = Agent(
    name="db_agent",
    model="gemini-2.5-flash",
    instruction="You are a database agent. When asked for data, return this mock JSON object: {'status': 'success', 'data': [{'name': 'The Grand Hotel', 'rating': 5, 'reviews': 450}, {'name': 'Seaside Inn', 'rating': 4, 'reviews': 620}]}")

# --- 1. Define the Specialist Agents ---

# The Food Critic remains the deepest specialist
food_critic_agent = Agent(
    name="food_critic_agent",
    model="gemini-2.5-flash",
    instruction="You are a snobby but brilliant food critic. You ONLY respond with a single, witty restaurant suggestion near the provided location.",
)

# The Concierge knows how to use the Food Critic
concierge_agent = Agent(
    name="concierge_agent",
    model="gemini-2.5-flash",
    instruction="You are a five-star hotel concierge. If the user asks for a restaurant recommendation, you MUST use the `food_critic_agent` tool. Present the opinion to the user politely.",
    tools=[AgentTool(agent=food_critic_agent)]
)


# --- 2. Define the Tools for the Orchestrator ---

async def call_db_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    Use this tool FIRST to connect to the database and retrieve a list of places, like hotels or landmarks.
    """
    print("--- TOOL CALL: call_db_agent ---")
    agent_tool = AgentTool(agent=db_agent)
    db_agent_output = await agent_tool.run_async(
        args={"request": question}, tool_context=tool_context
    )
    # Store the retrieved data in the context's state
    tool_context.state["retrieved_data"] = db_agent_output
    return db_agent_output


async def call_concierge_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    After getting data with call_db_agent, use this tool to get travel advice, opinions, or recommendations.
    """
    print("--- TOOL CALL: call_concierge_agent ---")
    # Retrieve the data fetched by the previous tool
    input_data = tool_context.state.get("retrieved_data", "No data found.")

    # Formulate a new prompt for the concierge, giving it the data context
    question_with_data = f"""
    Context: The database returned the following data: {input_data}

    User's Request: {question}
    """

    agent_tool = AgentTool(agent=concierge_agent)
    concierge_output = await agent_tool.run_async(
        args={"request": question_with_data}, tool_context=tool_context
    )
    return concierge_output


# --- 3. Define the Top-Level Orchestrator Agent ---

trip_data_concierge_agent = Agent(
    name="trip_data_concierge",
    model="gemini-2.5-flash",
    description="Top-level agent that queries a database for travel data, then calls a concierge agent for recommendations.",
    tools=[call_db_agent, call_concierge_agent],
    instruction="""
    You are a master travel planner who uses data to make recommendations.

    1.  **ALWAYS start with the `call_db_agent` tool** to fetch a list of places (like hotels) that match the user's criteria.

    2.  After you have the data, **use the `call_concierge_agent` tool** to answer any follow-up questions for recommendations, opinions, or advice related to the data you just found.
    """,
)

print(f"✅ Orchestrator Agent '{trip_data_concierge_agent.name}' is defined and ready.")

✅ Orchestrator Agent 'trip_data_concierge' is defined and ready.


In [13]:
# --- Let's test the Trip Data Concierge Agent ---

async def run_trip_data_concierge():
    """
    Sets up a session and runs a query against the top-level
    trip_data_concierge_agent.
    """
    # Create a new, single-use session for this query
    concierge_session = await session_service.create_session(
        app_name=trip_data_concierge_agent.name,
        user_id=my_user_id
    )

    # This query is specifically designed to trigger the full two-step process:
    # 1. Get data from the db_agent.
    # 2. Get a recommendation from the concierge_agent based on that data.
    # query = "Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews."
    query = "Find the top-rated hotels in Bangkok from the database"
    print(f"🗣️ User Query: '{query}'")

    # We call our existing helper function with the top-level orchestrator agent
    await run_agent_query(trip_data_concierge_agent, query, concierge_session, my_user_id)

# Run the test
await run_trip_data_concierge()

🗣️ User Query: 'Find the top-rated hotels in Bangkok from the database'

🚀 Running query for agent: 'trip_data_concierge' in session: '23718d95-05c3-4f2e-93e6-2280d322f45c'...


EVENT: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'question': 'Find the top-rated hotels in Bangkok'
        },
        id='adk-d57e40af-233a-47e0-826c-51a07609c832',
        name='call_db_agent'
      ),
      thought_signature=b'\n\xab\x02\x01\xd1\xed\x8ao]\xa4\x05\x82\x0cwe\xd1\xb7\x9d \x82TR\xeattD\x98\xe7\xeaR1\xc1\xd0v\x8e\x07\x19c\x13\xa6\xe4D=\xf93\xb9\xbe\x8d\x95\x0b#\x1e\x8a\x0c\xab\x0eU\x8b\xe73k\xc39a\xc6\x87\x85\xa7\xb4,"t\x92O!a\xd4\x12\x86UzREX\xf6\xa8mKh\x06\x86\xac6\x1ba\xd9c...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=24,
  prompt_token_count=278,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=278
    ),
  ],
  though

EVENT: content=Content(
  parts=[
    Part(
      text="""I couldn't filter the hotels by Bangkok since I don't have location data. However, here are the top-rated hotels from my records:

The Grand Hotel with a rating of 5 and 450 reviews.
Seaside Inn with a rating of 4 and 620 reviews.""",
      thought_signature=b'\n\xde\x03\x01\xd1\xed\x8ao"\xcd\x96S\x1d@\xb1N\xdb\xc1l\xa7\xb8Y\x84\xcc\xe2uE!\xf1\x8ax\xd6\x9a\x1eL\xe5\x90\r\xaf[\xce\xaey\xc5a\'&^\xbd\x92\xa5t\xe3\xf5pB\x92\xb7Gw\rj\x11,\x18f$,\xc6\x97w{\xfc\xdd\x0b\xab\xde\x03\xeb\xda\n\x01\x9b\x1a\xfa\xff~\x17xl1V\xebqA\xcb\xe2...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=65,
  prompt_token_count=428,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
   

I couldn't filter the hotels by Bangkok since I don't have location data. However, here are the top-rated hotels from my records:

The Grand Hotel with a rating of 5 and 450 reviews.
Seaside Inn with a rating of 4 and 620 reviews.

--------------------------------------------------



---
## Part 3: Agent with a Memory - The Adaptive Planner 🗺️

Now, let's see an agent that not only **remembers** but also **adapts**. We'll challenge the `multi_day_trip_agent` to re-plan part of its itinerary based on our feedback. This is a much more realistic test of conversational AI.

```
+-----------------------------------------------------+
|         Adaptive Multi-Day Trip Agent 🗺️           |
|-----------------------------------------------------|
|  Model: gemini-2.5-flash                            |
|  Description:                                       |
|   Builds multi-day travel itineraries step-by-step, |
|   remembers previous days, adapts to feedback       |
|-----------------------------------------------------|
|  🔧 Tools:                                          |
|   - Google Search                                   |
|-----------------------------------------------------|
|  🧠 Capabilities:                                   |
|   - Memory of past conversation & preferences       |
|   - Progressive planning (1 day at a time)          |
|   - Adapts to user feedback                         |
|   - Ensures activity variety across days            |
+-----------------------------------------------------+

            ▲
            |
    +---------------------------+
    |     User Interaction      |
    |---------------------------|
    | - Destination             |
    | - Trip duration           |
    | - Interests & feedback    |
    +---------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Day-by-Day Itinerary Generation              |
|-----------------------------------------------------|
|  🗓️ Day N Output (Markdown format):                 |
|   - Morning / Afternoon / Evening activities        |
|   - Personalized & context-aware                    |
|   - Changes accepted, feedback acknowledged         |
+-----------------------------------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Next Day Planning Triggered 🚀               |
|-----------------------------------------------------|
| - Builds on prior days                              |
| - Avoids repetition                                 |
| - Asks user for confirmation before proceeding      |
+-----------------------------------------------------+
```


In [14]:
# --- Agent Definition: The Adaptive Planner ---

def create_multi_day_trip_agent():
    """Create the Progressive Multi-Day Trip Planner agent"""
    return Agent(
        name="multi_day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent that progressively plans a multi-day trip, remembering previous days and adapting to user feedback.",
        instruction="""
        You are the "Adaptive Trip Planner" 🗺️ - an AI assistant that builds multi-day travel itineraries step-by-step.

        Your Defining Feature:
        You have short-term memory. You MUST refer back to our conversation to understand the trip's context, what has already been planned, and the user's preferences. If the user asks for a change, you must adapt the plan while keeping the unchanged parts consistent.

        Your Mission:
        1.  **Initiate**: Start by asking for the destination, trip duration, and interests.
        2.  **Plan Progressively**: Plan ONLY ONE DAY at a time. After presenting a plan, ask for confirmation.
        3.  **Handle Feedback**: If a user dislikes a suggestion (e.g., "I don't like museums"), acknowledge their feedback, and provide a *new, alternative* suggestion for that time slot that still fits the overall theme.
        4.  **Maintain Context**: For each new day, ensure the activities are unique and build logically on the previous days. Do not suggest the same things repeatedly.
        5.  **Final Output**: Return each day's itinerary in MARKDOWN format.
        """,
        tools=[google_search]
    )

multi_day_agent = create_multi_day_trip_agent()
print(f"🗺️ Agent '{multi_day_agent.name}' is created and ready to plan and adapt!")

🗺️ Agent 'multi_day_trip_agent' is created and ready to plan and adapt!


### Scenario 3a: Agent WITH Memory (Using a SINGLE Session) ✅

First, let's see the correct way to do it. We will use the **exact same `trip_session` object** for the entire conversation. Watch how the agent remembers the context from Turn 1 to correctly handle the requests in Turn 2 and 3.

In [15]:
# --- Scenario 2: Testing Adaptation and Memory ---

async def run_adaptive_memory_demonstration():
    print("### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###")

    # Create ONE session that we will reuse for the whole conversation
    trip_session = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"Created a single session for our trip: {trip_session.id}")

    # --- Turn 1: The user initiates the trip ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    print(f"\n🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, trip_session, my_user_id)

    # --- Turn 2: The user gives FEEDBACK and asks for a CHANGE ---
    # We use the EXACT SAME `trip_session` object!
    query2 = "That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?"
    print(f"\n🗣️ User (Turn 2 - Feedback): '{query2}'")
    await run_agent_query(multi_day_agent, query2, trip_session, my_user_id)

    # --- Turn 3: The user confirms and asks to continue ---
    query3 = "Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind."
    print(f"\n🗣️ User (Turn 3 - Confirmation): '{query3}'")
    await run_agent_query(multi_day_agent, query3, trip_session, my_user_id)

await run_adaptive_memory_demonstration()

### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###
Created a single session for our trip: dea8b973-961b-4a5c-85da-ea9131cb5f03

🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'dea8b973-961b-4a5c-85da-ea9131cb5f03'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Hello! I'd love to help you plan your 2-day trip to Lisbon, focusing on historic sites and delicious local food. Lisbon is a fantastic choice for both!

Let's start with Day 1. How does this sound for your first day in Lisbon?

### **Day 1: Age of Discovery & Alfama Charm**

*   **Morning (9:00 AM - 1:00 PM): Belém's Historic Treasures**
    Start your day in the historic Belém district, a UNESCO World Heritage site and a testament to Portugal's Age of Discovery.
    *   **Jerónimos Monastery (Mosteiro dos Jerónimos):** Explore this magnificent example of Manueline architect

Hello! I'd love to help you plan your 2-day trip to Lisbon, focusing on historic sites and delicious local food. Lisbon is a fantastic choice for both!

Let's start with Day 1. How does this sound for your first day in Lisbon?

### **Day 1: Age of Discovery & Alfama Charm**

*   **Morning (9:00 AM - 1:00 PM): Belém's Historic Treasures**
    Start your day in the historic Belém district, a UNESCO World Heritage site and a testament to Portugal's Age of Discovery.
    *   **Jerónimos Monastery (Mosteiro dos Jerónimos):** Explore this magnificent example of Manueline architecture, admiring its intricate cloisters and grand church. It houses the tomb of Vasco da Gama.
    *   **Belém Tower (Torre de Belém):** A 16th-century fortification that once guarded Lisbon's harbor. You can admire its unique Manueline style from the outside and enjoy views of the Tagus River.
    *   **Pastéis de Belém:** Indulge in the original, warm custard tarts at the iconic Pastéis de Belém bakery, whose secret recipe has remained unchanged since 1837.

*   **Lunch (1:00 PM - 2:30 PM): Authentic Portuguese Bites**
    Grab a traditional Portuguese lunch in the Belém area. Perhaps try a "Bifana," a delicious pork sandwich, or "Pastéis de Bacalhau" (fried codfish cakes).

*   **Afternoon (2:30 PM - 6:30 PM): Alfama's Medieval Labyrinth**
    Head to the oldest district of Lisbon, Alfama, known for its narrow, winding streets and medieval charm.
    *   **São Jorge Castle (Castelo de São Jorge):** Explore this ancient Moorish castle perched atop a hill, offering panoramic views of the city's red rooftops and the Tagus River.
    *   **Wander through Alfama:** Get lost in the labyrinthine alleys, discover hidden viewpoints (miradouros), and soak in the historic atmosphere.

*   **Evening (7:00 PM onwards): Traditional Flavors & Fado**
    Enjoy a traditional Portuguese dinner in the Alfama district. You could try "Bacalhau à Brás" (shredded cod with potatoes and eggs) or "Sardinhas Assadas" (grilled sardines, especially popular in summer). If you're interested, consider finding a restaurant that offers a Fado performance, Lisbon's soulful traditional music.

How does this first day sound to you? We can adjust anything you like!

--------------------------------------------------


🗣️ User (Turn 2 - Feedback): 'That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'dea8b973-961b-4a5c-85da-ea9131cb5f03'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Understood! My apologies for including a castle in the morning when you're not a fan. You're right, São Jorge Castle is in the afternoon, but I can definitely revise the morning for Day 1 to offer a different historical experience.

Let's try this revised plan for your first day:

### **Day 1: Lisbon's Resilience & Alfama Charm**

*   **Morning (9:00 AM - 1:00 PM): Carmo Convent & Baixa Exploration**
    Start your day exploring the heart of Lisbon, witnessing its resilience after the devastating 1755 earthquake.
    *   **Santa Justa Lift (Elevador de Santa Justa):** Take this iconic iron elevator (designe

Understood! My apologies for including a castle in the morning when you're not a fan. You're right, São Jorge Castle is in the afternoon, but I can definitely revise the morning for Day 1 to offer a different historical experience.

Let's try this revised plan for your first day:

### **Day 1: Lisbon's Resilience & Alfama Charm**

*   **Morning (9:00 AM - 1:00 PM): Carmo Convent & Baixa Exploration**
    Start your day exploring the heart of Lisbon, witnessing its resilience after the devastating 1755 earthquake.
    *   **Santa Justa Lift (Elevador de Santa Justa):** Take this iconic iron elevator (designed by an apprentice of Gustave Eiffel) for impressive panoramic views of the city, connecting the Baixa district with the Bairro Alto.
    *   **Carmo Convent (Convento do Carmo):** Explore the hauntingly beautiful ruins of this Gothic church, largely destroyed in the earthquake. It now houses an archaeological museum and is a powerful reminder of Lisbon's past.
    *   **Wander through Baixa:** Stroll through the grid-patterned streets of the Baixa district, admiring the Pombaline architecture developed during its post-earthquake reconstruction. Head towards **Praça do Comércio**, a grand waterfront square on the Tagus River.

*   **Lunch (1:00 PM - 2:30 PM): Traditional Flavors in Baixa**
    Enjoy a traditional Portuguese lunch in the Baixa district. You'll find many restaurants offering classic dishes like "Bacalhau à Brás" (shredded cod with potatoes and eggs) or fresh seafood.

*   **Afternoon (2:30 PM - 6:30 PM): Alfama's Medieval Labyrinth**
    Head to the oldest district of Lisbon, Alfama, known for its narrow, winding streets and medieval charm.
    *   **São Jorge Castle (Castelo de São Jorge):** Explore this ancient Moorish castle perched atop a hill, offering panoramic views of the city's red rooftops and the Tagus River.
    *   **Wander through Alfama:** Get lost in the labyrinthine alleys, discover hidden viewpoints (miradouros), and soak in the historic atmosphere.

*   **Evening (7:00 PM onwards): Traditional Flavors & Fado**
    Enjoy a traditional Portuguese dinner in the Alfama district. You could try "Sardinhas Assadas" (grilled sardines, especially popular in summer) or other local specialties. If you're interested, consider finding a restaurant that offers a Fado performance, Lisbon's soulful traditional music.

How does this revised Day 1 sound for you? We can still adjust anything else if needed!

--------------------------------------------------


🗣️ User (Turn 3 - Confirmation): 'Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: 'dea8b973-961b-4a5c-85da-ea9131cb5f03'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Excellent! I'm glad Day 1 is perfect. Let's make Day 2 just as wonderful, keeping your interests in historic sites and delicious local food at the forefront.

Here is a plan for your second day in Lisbon:

### **Day 2: Pantheon Views, Culinary Wonders & Artistic Heritage**

*   **Morning (9:00 AM - 1:00 PM): Grand Monuments & Panoramic Views**
    Start your day exploring the beautiful São Vicente area, offering both grand history and stunning cityscapes.
    *   **National Pantheon (Panteão Nacional):** Visit this impressive monument, originally a church, now the final resting place for many important Portuguese figures. Admire its majestic inte

Excellent! I'm glad Day 1 is perfect. Let's make Day 2 just as wonderful, keeping your interests in historic sites and delicious local food at the forefront.

Here is a plan for your second day in Lisbon:

### **Day 2: Pantheon Views, Culinary Wonders & Artistic Heritage**

*   **Morning (9:00 AM - 1:00 PM): Grand Monuments & Panoramic Views**
    Start your day exploring the beautiful São Vicente area, offering both grand history and stunning cityscapes.
    *   **National Pantheon (Panteão Nacional):** Visit this impressive monument, originally a church, now the final resting place for many important Portuguese figures. Admire its majestic interior and climb to the dome for breathtaking panoramic views over Lisbon and the Tagus River.
    *   **Monastery of São Vicente de Fora (Mosteiro de São Vicente de Fora):** Just a short walk from the Pantheon, explore this historic 16th-century monastery. It's renowned for its beautiful cloisters adorned with magnificent azulejos (traditional Portuguese tiles) depicting Fables of La Fontaine, and its royal pantheon.

*   **Lunch (1:00 PM - 2:30 PM): A Feast at Time Out Market**
    Head to the vibrant **Time Out Market (Mercado da Ribeira)**. This bustling food hall is a paradise for food lovers, bringing together some of Lisbon's best chefs and restaurants under one roof. You can sample a wide array of Portuguese and international dishes, from traditional petiscos (tapas) to gourmet meals, all in one lively spot.

*   **Afternoon (2:30 PM - 6:30 PM): Elegant Streets & Tile Artistry**
    *   **Chiado District:** Stroll through the elegant and sophisticated Chiado district, known for its historic theaters, traditional bookstores, and charming cafes like "A Brasileira." It offers a different historical and cultural atmosphere than Alfama, with its 18th-century charm.
    *   **National Azulejo Museum (Museu Nacional do Azulejo):** Take a short taxi or public transport ride to this unique museum, housed in the beautiful former Convent of Madre de Deus. It provides a fascinating journey through the history of Portuguese azulejos (ceramic tiles) from the 15th century to the present, showcasing an integral part of Portugal's artistic and architectural heritage.

*   **Evening (7:00 PM onwards): Farewell Dinner in Principe Real**
    Enjoy a relaxed farewell dinner in the charming **Principe Real** neighborhood. Known for its beautiful gardens, independent boutiques, and diverse culinary scene, it offers many excellent restaurant choices, from modern Portuguese cuisine to international flavors, for a memorable end to your trip.

How does this plan for Day 2 sound for you?

--------------------------------------------------



### Scenario 3b: Agent WITHOUT Memory (Using SEPARATE Sessions) ❌

Now, let's see what happens if we mess up our session management. Here, we'll give the agent a case of amnesia by creating a **brand new, separate session for each turn**.

Pay close attention to the agent's response to the second query. Because it's in a new session, it has no memory of the trip to Lisbon we just discussed!

In [16]:
# --- Scenario 2b: Demonstrating Memory FAILURE ---

async def run_memory_failure_demonstration():
    print("\n" + "#"*60)
    print("### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###")
    print("#"*60)

    # --- Turn 1: The user initiates the trip in the FIRST session ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    session_one = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a session for Turn 1: {session_one.id}")
    print(f"🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, session_one, my_user_id)

    # --- Turn 2: The user asks to continue... but in a completely NEW session ---
    query2 = "Yes, that looks perfect! Please plan Day 2."
    session_two = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a BRAND NEW session for Turn 2: {session_two.id}")
    print(f"🗣️ User (Turn 2): '{query2}'")
    await run_agent_query(multi_day_agent, query2, session_two, my_user_id)

await run_memory_failure_demonstration()


############################################################
### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###
############################################################

Created a session for Turn 1: 3f4c9a55-4a3c-4491-a749-3cde9b05bcb0
🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '3f4c9a55-4a3c-4491-a749-3cde9b05bcb0'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Great! Lisbon is a fantastic choice for history and food lovers. We'll plan a wonderful 2-day itinerary for you.

Let's start with **Day 1**. How does this sound?

### Day 1: Alfama's Charms & Culinary Delights

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District & Lisbon Cathedral**
    *   Begin your day by wandering through the labyrinthine streets of Alfama, Lisbon's oldest district. Discover hidden alleys, colorful houses, and stunning viewpoin

Great! Lisbon is a fantastic choice for history and food lovers. We'll plan a wonderful 2-day itinerary for you.

Let's start with **Day 1**. How does this sound?

### Day 1: Alfama's Charms & Culinary Delights

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District & Lisbon Cathedral**
    *   Begin your day by wandering through the labyrinthine streets of Alfama, Lisbon's oldest district. Discover hidden alleys, colorful houses, and stunning viewpoints.
    *   Visit the Lisbon Cathedral (Sé de Lisboa), a national monument with a rich history dating back to the 12th century.
*   **Lunch (1:00 PM - 2:30 PM): Authentic Portuguese Lunch in Alfama**
    *   Enjoy a traditional Portuguese lunch at a local tasca (tavern) in Alfama, trying dishes like *bacalhau à brás* (shredded cod with eggs and potatoes) or *sardinhas assadas* (grilled sardines).
*   **Afternoon (2:30 PM - 6:00 PM): São Jorge Castle & Miradouro da Senhora do Monte**
    *   Head up to São Jorge Castle (Castelo de São Jorge), an iconic historic castle offering panoramic views over the city and the Tagus River. Explore its ramparts, gardens, and archaeological site.
    *   Afterwards, make your way to Miradouro da Senhora do Monte (Our Lady of the Hill Viewpoint) for one of the most breathtaking sunset views in Lisbon.
*   **Evening (7:30 PM onwards): Fado Show & Dinner in Mouraria**
    *   Experience an authentic Fado show in the historic Mouraria district, often considered the birthplace of Fado. Enjoy traditional Portuguese cuisine while listening to the melancholic and soulful music.

How does this sound for your first day in Lisbon?

--------------------------------------------------


Created a BRAND NEW session for Turn 2: 42d41bba-97a9-4e75-a84a-fd33f4d1ca7e
🗣️ User (Turn 2): 'Yes, that looks perfect! Please plan Day 2.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '42d41bba-97a9-4e75-a84a-fd33f4d1ca7e'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Here's a plan for Day 2 in New York City:

## Day 2: Lower Manhattan History & Views

*   **Morning (9:00 AM - 1:00 PM): Statue of Liberty & Ellis Island**
    *   Start your day by taking the ferry from Battery Park to Liberty Island to see the Statue of Liberty up close.
    *   Continue to Ellis Island to explore the Ellis Island National Museum of Immigration and learn about the immigrant experience.
*   **Lunch (1:00 PM - 2:00 PM): Financial District Bites**
    *   Grab a quick and delicious lunch in the Financial District. There are many options, from casual delis to upscale eateries.
*   **Afternoon (2:00 PM - 5:00 PM): 9/11 Me

Here's a plan for Day 2 in New York City:

## Day 2: Lower Manhattan History & Views

*   **Morning (9:00 AM - 1:00 PM): Statue of Liberty & Ellis Island**
    *   Start your day by taking the ferry from Battery Park to Liberty Island to see the Statue of Liberty up close.
    *   Continue to Ellis Island to explore the Ellis Island National Museum of Immigration and learn about the immigrant experience.
*   **Lunch (1:00 PM - 2:00 PM): Financial District Bites**
    *   Grab a quick and delicious lunch in the Financial District. There are many options, from casual delis to upscale eateries.
*   **Afternoon (2:00 PM - 5:00 PM): 9/11 Memorial & Wall Street**
    *   Visit the solemn 9/11 Memorial & Museum to pay respects and learn about the events of September 11, 2001.
    *   Walk through the historic streets of the Financial District, see the New York Stock Exchange, and rub the Charging Bull for good luck.
*   **Evening (6:00 PM onwards): Dinner in Tribeca & Waterfront Stroll**
    *   Enjoy dinner in Tribeca, known for its trendy restaurants and cobblestone streets.
    *   After dinner, take a leisurely stroll along the Hudson River waterfront for beautiful views of the city skyline and New Jersey.

How does Day 2 sound to you?

--------------------------------------------------



See? The agent was confused! It likely asked what destination or what trip we were talking about. Because the second query was in a fresh, isolated session, the agent had no memory of planning Day 1 in Lisbon.

This perfectly illustrates why **managing sessions is the key to building truly conversational agents!**

---
## 🎉 Congratulations! 🎉

Congratulations on completing your ADK adventure into Tools and Memory! You've taken a massive leap from building single-shot agents to creating dynamic, stateful AI systems.

Let's recap the powerful concepts you've mastered:

- **Fundamental Agent & Tools**: You started by building a "Day Trip Genie" and equipped it with its first tool, GoogleSearch.

- **Custom Function Tools**: You gave your agent a new sense by creating a custom tool to fetch live data from the U.S. National Weather Service API.

- **Agent-as-a-Tool**: You orchestrated a sophisticated hierarchy where agents delegate tasks to other, more specialized agents, creating a collaborative team.

- **The Power of Memory**: Most importantly, you saw firsthand how managing a single, persistent Session allows an agent to remember context, adapt to user feedback, and conduct a meaningful, multi-turn conversation.

```
   __            /\_/\         /\_/\        /\_/\         __             (\__/)
o-''|\_____/).  ( o.o )       ( -.- )      ( ^_^ )     o-''|\_____/).    ( ^_^ )
 \_/|_)     )    > ^ <         > * <        >💖<         \_/|_)     )     / >🌸< \
    \  __  /                                              \  __  /         /   \
    (_/ (_/                                               (_/ (_/        (___|___)
```
